In [1]:
import json
import numpy as np
import pandas as pd
from monty.serialization import loadfn, dumpfn
from glob import glob
from datetime import datetime
%env BEEP_ENV=dev
from beep import structure, run_model, featurize
from beep.featurize import (
    RPTdQdVFeatures,
    HPPCResistanceVoltageFeatures,
    DiagnosticSummaryStats,
    HPPCRelaxationFeatures,
    DiagnosticProperties
)

# import yaml
from beep.dataset import BeepDataset
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
import seaborn as sns
import os
# from throughput_dev_tri import get_threshold_targets
import pickle
from scipy.interpolate import interp1d
from scipy import optimize
from scipy.optimize import differential_evolution
# from dtw import *
from IntracellAnalysis import *
from test_time_helpers import *

env: BEEP_ENV=dev


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)

In [3]:
# Setting Path for Loading Data in Loop
project_list = ["PreDiag", "PredictionDiagnostics"]
FEATURIZER_CLASSES = [RPTdQdVFeatures, HPPCResistanceVoltageFeatures, DiagnosticSummaryStats, DiagnosticProperties]

raw_path = "/home/ec2-user/SageMaker/efs-readonly/raw/maccor"
processed_path = "/home/ec2-user/SageMaker/efs-readonly/structure/"
featurized_path = "/home/ec2-user/SageMaker/efs-readonly/features/"
predictions_path = "/home/ec2-user/SageMaker/efs-readonly/features/"

## Function call to load in test times of all cells past threshold with specified cycling parameters

In [8]:
protocol_params = pd.DataFrame({'charge_constant_current_1' : 0.5,
                                'charge_constant_current_2' : 0.5,
                                'discharge_constant_current' : 0.2,
                                'charge_cutoff_voltage' : 4.1,
                                'discharge_cutoff_voltage' : 2.7},index=[0])
test_time_df, test_time_stats_df = get_protocol_test_time_stats(protocol_params=protocol_params)

In [9]:
test_time_df

,test_time_to_eol
174,1.419632e+07
177,1.190680e+07
256,1.192188e+07
359,1.187580e+07
360,1.188340e+07
361,1.179358e+07


In [11]:
test_time_stats_df

,mean,std
0,1.226296e+07,865577.677997


## Functional call for test time based on provided list of seq_num

In [14]:
test_time_df = get_test_time_df_from_seq_num_list(seq_num_list=[174,177],
                                       nominal_capacity=4.83,
                                       threshold=0.8,
                                       cycle_type='rpt_0.2C',
                                       project_list=["PreDiag", "PredictionDiagnostics"],
                                       processed_path="/home/ec2-user/SageMaker/efs-readonly/structure/")

In [15]:
test_time_df

,test_time_to_eol
174,1.419632e+07
177,1.190680e+07


## Function call for cell that is unfinished

In [4]:
protocol_params = pd.DataFrame({'charge_constant_current_1' : 0.2,
                                'charge_constant_current_2' : 0.2,
                                'discharge_constant_current' : 0.2,
                                'charge_cutoff_voltage' : 4.1,
                                'discharge_cutoff_voltage' : 3.7},index=[0])
unfinished_test_time_df = get_unfinished_test_time_from_seq_num_list(seq_num_list=[246],
                                         nominal_capacity=4.83,
                                         threshold=0.8,
                                         cycle_type='rpt_0.2C'
                                         )

In [5]:
unfinished_test_time_df/60/60/24

,test_time_to_date
246,260.600066


In [181]:
test_time_df/60/60/24
test_time_stats_df['mean']/60/60/24

0    117.072237
Name: mean, dtype: float64